# Option 1. Initialize without telemetry

* For first cluster

In [ ]:
%%bash
terraform output -raw cluster1_init_remote
eval "$(terraform output -raw cluster1_init_remote)"

* For second cluster

In [ ]:
%%bash
terraform output -raw cluster2_init_remote
eval "$(terraform output -raw cluster2_init_remote)"

In [ ]:
%%bash
terraform destroy -var-file=variables.tfvars --auto-approve

# Option 2. Initialize with telemetry

* Initialize the first cluster. Requires SSH access via console.
* Enable Audit logging
* Obtain a token and configure the agent to use it

In [ ]:
%%bash
terraform output -raw cluster1_init_script_node1

* Obtain token for Ops Agent
* Restart Ops Agent Service
* Repeat on nodes 2 to X

In [ ]:
%%bash
terraform output -raw cluster1_init_script_node2-X

* Enable Automatic Backups on local terminal

In [ ]:
%%bash
terraform output -raw cluster1_init_auto_snapshot

* Connect to cluster

In [ ]:
%%bash
terraform output -raw cluster1_fqdn_8200

* Repeat for other clusters

In [ ]:
%%bash
terraform output -raw cluster2_init_script_node1

In [ ]:
%%bash
terraform output -raw cluster2_init_script_node2-X

In [ ]:
%%bash
terraform output -raw cluster2_init_auto_snapshot

* Connect to cluster

In [ ]:
%%bash
terraform output -raw cluster2_fqdn_8200